In [1]:
from os import walk
from PIL import Image
import numpy as np
from tqdm import tqdm

EPSILON = 10**(-20)

In [2]:


filenames = next(walk('dataset/support_96'), (None, None, []))[2]  # [] if no file
len(filenames)

200

In [3]:
img1 = np.asarray(Image.open('dataset/support_96/Acadian_Flycatcher_0016_887710060.jpg'))
img2 = np.asarray(Image.open('dataset/query_1/Acadian_Flycatcher_0016_887710060.jpg'))
img3 = np.asarray(Image.open('dataset/query_1/American_Crow_0024_2618947526.jpg'))


In [4]:
def per_channel_histogram(img, interval=1):
    x , y = img.shape[0], img.shape[1]
    
    histograms = {
        'red':np.zeros(256, np.int16), 
        'green':np.zeros(256, np.int16), 
        'blue':np.zeros(256, np.int16)
        }

    for i in range(x):
        for j in range(y):
            histograms['red'][img[i][j][0]] += 1
            histograms['green'][img[i][j][1]] += 1
            histograms['blue'][img[i][j][2]] += 1

    if interval == 1:
        return histograms
    
    bin_num = 256 // interval
    
    new_histograms = {
        'red':np.zeros(bin_num, np.int16), 
        'green':np.zeros(bin_num, np.int16), 
        'blue':np.zeros(bin_num, np.int16)
        }

    for i in range(bin_num):
        for j in range(interval):
            new_histograms['red'][i] += histograms['red'][i * interval + j]
            new_histograms['green'][i] += histograms['green'][i * interval + j]
            new_histograms['blue'][i] += histograms['blue'][i * interval + j]

    return new_histograms
    

In [5]:
def normalize_histogram(h):
    norm = np.sum(h)
    normalized_hist = h/np.sum(h)
    for i in range(normalized_hist.shape[0]):
        if normalized_hist[i] == 0:
            normalized_hist[i] = EPSILON / norm
    return normalized_hist

def kl_divergence(query_hist, support_hist):
    total = 0
    for q, s in zip(query_hist, support_hist):
        total += q * np.log2(q/s)
    return total

In [6]:
hist1 = per_channel_histogram(img1)
hist2 = per_channel_histogram(img2)
red = kl_divergence(normalize_histogram(hist1['red']), normalize_histogram(hist2['red']))
print(red)

0.3801293466459517


In [7]:
support_filenames = next(walk('dataset/support_96'), (None, None, []))[2]
query_1_filenames = next(walk('dataset/query_1'), (None, None, []))[2]
query_2_filenames = next(walk('dataset/query_2'), (None, None, []))[2]
query_3_filenames = next(walk('dataset/query_3'), (None, None, []))[2]

support_images = []
for filename in support_filenames:
    with Image.open('dataset/support_96/{}'.format(filename)) as image:
        support_images.append((filename, np.asarray(image)))

query_1_images = []
for filename in query_1_filenames:
    with Image.open('dataset/query_1/{}'.format(filename)) as image:
        query_1_images.append((filename, np.asarray(image)))
query_2_images = []
for filename in query_2_filenames:
    with Image.open('dataset/query_2/{}'.format(filename)) as image:
        query_2_images.append((filename, np.asarray(image)))

query_3_images = []
for filename in query_3_filenames:
    with Image.open('dataset/query_3/{}'.format(filename)) as image:
        query_3_images.append((filename, np.asarray(image)))

In [8]:
# report 2
# for q1
q1_min_results = []
intervals = [1,2,4,8,16]
q1_accuracies = []
for q in intervals:
    correct_guesses = 0
    for name, img in tqdm(support_images, desc="support images", position=0):
        min_divergence = 99
        hist = per_channel_histogram(img, interval=q)
        for q_name, q_img in query_1_images:
            q_hist = per_channel_histogram(q_img, interval=q)
            red_divergence = kl_divergence(normalize_histogram(q_hist['red']), normalize_histogram(hist['red']))
            green_divergence = kl_divergence(normalize_histogram(q_hist['green']), normalize_histogram(hist['green']))
            blue_divergence = kl_divergence(normalize_histogram(q_hist['blue']), normalize_histogram(hist['blue']))
            divergence = (red_divergence + blue_divergence + blue_divergence) / 3
            if divergence < min_divergence:
                min_divergence = divergence
                result = {'support': name, 'query': q_name, 'divergence': divergence}
        q1_min_results.append(result)
        if result['support'] == result['query']:
            correct_guesses += 1
    q1_accuracies.append({'interval': q, 'acc': correct_guesses / 200})

support images:  42%|████▏     | 84/200 [18:36<25:55, 13.41s/it]

In [ ]:
with open("q1_acc.txt", 'w') as f:
    f.write(str(q1_min_results))
    f.write('\n\n')
    f.write(str(q1_accuracies))

In [ ]:
# for q2
q2_min_results = []
intervals = [1,2,4,8,16]
q2_accuracies = []
for q in intervals:
    correct_guesses = 0
    for name, img in tqdm(support_images, desc="support images", position=0):
        min_divergence = 99
        hist = per_channel_histogram(img, interval=q)
        for q_name, q_img in query_2_images:
            q_hist = per_channel_histogram(q_img, interval=q)
            red_divergence = kl_divergence(normalize_histogram(q_hist['red']), normalize_histogram(hist['red']))
            green_divergence = kl_divergence(normalize_histogram(q_hist['green']), normalize_histogram(hist['green']))
            blue_divergence = kl_divergence(normalize_histogram(q_hist['blue']), normalize_histogram(hist['blue']))
            divergence = (red_divergence + blue_divergence + blue_divergence) / 3
            if divergence < min_divergence:
                min_divergence = divergence
                result = {'support': name, 'query': q_name, 'divergence': divergence}
        q2_min_results.append(result)
        if result['support'] == result['query']:
            correct_guesses += 1
    q2_accuracies.append({'interval': q, 'acc': correct_guesses / 200})

In [ ]:
with open("q2_acc.txt", 'w') as f:
    f.write(str(q2_min_results))
    f.write('\n\n')
    f.write(str(q2_accuracies))

In [ ]:
# for q3
q3_min_results = []
intervals = [1,2,4,8,16]
q3_accuracies = []
for q in intervals:
    correct_guesses = 0
    for name, img in tqdm(support_images, desc="support images", position=0):
        min_divergence = 99
        hist = per_channel_histogram(img, interval=q)
        for q_name, q_img in query_3_images:
            q_hist = per_channel_histogram(q_img, interval=q)
            red_divergence = kl_divergence(normalize_histogram(q_hist['red']), normalize_histogram(hist['red']))
            green_divergence = kl_divergence(normalize_histogram(q_hist['green']), normalize_histogram(hist['green']))
            blue_divergence = kl_divergence(normalize_histogram(q_hist['blue']), normalize_histogram(hist['blue']))
            divergence = (red_divergence + blue_divergence + blue_divergence) / 3
            if divergence < min_divergence:
                min_divergence = divergence
                result = {'support': name, 'query': q_name, 'divergence': divergence}
        q3_min_results.append(result)
        if result['support'] == result['query']:
            correct_guesses += 1
    q3_accuracies.append({'interval': q, 'acc': correct_guesses / 200})

In [ ]:
with open("q3_acc.txt", 'w') as f:
    f.write(str(q3_min_results))
    f.write('\n\n')
    f.write(str(q3_accuracies))